In [1]:
import pandas as pd
import random
import re
from IPython.display import clear_output
import itertools

In [2]:
#create an index file for the huge data file
def createIndex(filePath):
    file = open(filePath,'r');
    indexFilePath = filePath+"_index.csv"
    indexFile = open(indexFilePath,'w');
    indexFile.close()
    
    offset = 0
    lineNumber = 0
    for line in file:
#         clear_output(wait=True)
#         print(lineNumber)
        indexFile = open(indexFilePath,'a');
        indexFile.write(str(lineNumber) + "," + str(offset) + "\n")
        indexFile.close()
        lineNumber = lineNumber + 1
        offset += len(line)
    file.close()
    return indexFilePath

In [3]:
def generateNIndecesFrom(n, rangeOfIndeces):
    print("Generating " + str(n) + " indeces from range")
    allIndeces = random.sample(rangeOfIndeces, n)
    allIndeces = pd.Series(data = allIndeces)
    allIndeces = allIndeces.sort_values().reset_index().drop(['index'],axis=1)
    allIndeces.columns = ['Index'];
    return allIndeces

In [4]:
#Load or Generate the 2M indeces to use
try:
    twoMIndeces = pd.read_csv('2MIndeces.csv',squeeze = True)
    
except:
    print("There were not 2000000 data points")
    twoMIndeces = generateNIndecesFrom(2000000,range(0,45840617)) # this range is because there are this number of records in the training set.
    twoMIndeces.to_csv('2MIndeces.csv')

In [11]:
def generateSubSet(file,dataFrame,indexValues,numRowsPerItteration,totalNumRows,column_headers,frameSaveName):
    totalNumIterations = int(totalNumRows/numRowsPerItteration)
    print("Number of itterations = " + str(totalNumIterations))
    totalNumRowsTraversed = 0
    prevsize = 0
    for i in range(totalNumIterations + 1):
#         
#         print("Itteration number: " + str(i))
#         print("skipRows: " + str(i * numRowsPerItteration))
#         print("Read in : " + str(numRowsPerItteration))
        curData = pd.read_table(file,skiprows = i * numRowsPerItteration, nrows = numRowsPerItteration,header=None)
        curData.index = [i for i in range(i*numRowsPerItteration,i*numRowsPerItteration + curData.shape[0])]
        totalNumRowsTraversed = totalNumRowsTraversed + curData.shape[0]

#         print(curData.shape)
#         print(curData.index.shape)

        
        curData.columns = column_headers
        curData['Index'] = curData.index
        
        curIndexRange = indexValues['Index'][(indexValues['Index'] < (i*numRowsPerItteration + numRowsPerItteration)) & (indexValues['Index'] > (i*numRowsPerItteration))]
        curData = curData[curData['Index'].isin(curIndexRange)]

        dataFrame = pd.concat([dataFrame,curData])
        
        clear_output()
        print("Extraction Stats: " + str(dataFrame.shape[0]) + " percent: " + str(dataFrame.shape[0] / indexValues.shape[0] * 100) + "%")
        print("Document Stats: " + str(totalNumRowsTraversed) + " percent: " + str(totalNumRowsTraversed/totalNumRows*100) + "%")
        if (dataFrame.shape[0] - prevsize) > 500000:
            prevsize = dataFrame.shape[0]
            dataFrame.to_csv(frameSaveName)
        elif dataFrame.shape[0] == indexValues.shape[0]:
            print("Finished with the data collection")
            dataFrame.to_csv(frameSaveName)
    print("Extraction is Done, now saving frame")        
    dataFrame.to_csv(frameSaveName)
    return dataFrame

In [12]:
column_headers = pd.read_csv('data_columns.csv',squeeze = True)
allData = pd.DataFrame()
allData = generateSubSet('dac/train.txt',allData,twoMIndeces,4000000,46000000,column_headers,'2MDataFinalDesktop.csv')

Extraction Stats: 1999999 percent: 99.99995%
Document Stats: 45840617 percent: 99.6535152173913%
Extraction is Done, now saving frame


In [55]:
allData = pd.read_csv('2MDataFinalDesktop.csv')

C:\Users\adriencogny\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
allData.head()

,Unnamed: 0,label,integer_1,integer_2,integer_3,integer_4,integer_5,integer_6,integer_7,integer_8,...,categorical_18,categorical_19,categorical_20,categorical_21,categorical_22,categorical_23,categorical_24,categorical_25,categorical_26,Index
0,4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,...,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN,4
1,14,0,0.0,51,84.0,4.0,3633.0,26.0,1.0,4.0,...,12195b22,21ddcdc9,b1252a9d,fa131867,NaN,dbb486d7,8ecc176a,e8b83407,c43c3f58,14
2,64,0,1.0,1,5.0,8.0,7.0,8.0,1.0,8.0,...,4b340164,NaN,NaN,NaN,NaN,3a171ecb,NaN,NaN,NaN,64
3,91,0,NaN,-1,NaN,NaN,23204.0,NaN,0.0,32.0,...,15a36060,NaN,NaN,0014c32a,NaN,423fab69,3b183c5c,NaN,NaN,91
4,117,1,NaN,-1,NaN,NaN,446.0,30.0,2.0,27.0,...,fffe2a63,21ddcdc9,b1252a9d,eb0fc6f8,NaN,32c7478e,df487a73,001f3601,c27f155b,117


In [37]:
allData.tail()

,Unnamed: 0,label,integer_1,integer_2,integer_3,integer_4,integer_5,integer_6,integer_7,integer_8,...,categorical_18,categorical_19,categorical_20,categorical_21,categorical_22,categorical_23,categorical_24,categorical_25,categorical_26,Index
1999994,45840529,0,NaN,417,57.0,1.0,11952.0,NaN,0.0,21.0,...,15a36060,NaN,NaN,0014c32a,NaN,bcdee96c,3b183c5c,NaN,NaN,45840529
1999995,45840572,0,NaN,0,1.0,1.0,18597.0,225.0,7.0,3.0,...,7e32f7a4,NaN,NaN,a4b7004c,NaN,3a171ecb,b34f3128,NaN,NaN,45840572
1999996,45840578,0,0.0,59,20.0,7.0,1945.0,120.0,7.0,36.0,...,9e4517be,3014a4b1,5840adea,572bdde8,NaN,55dd3565,9fa3e01a,001f3601,d9bcfc08,45840578
1999997,45840600,0,1.0,71,34.0,16.0,21.0,3.0,39.0,22.0,...,5bb2ec8e,d271d52f,b1252a9d,be2263d6,NaN,32c7478e,3fdb382b,e8b83407,49d68486,45840600
1999998,45840612,0,NaN,-1,NaN,NaN,34722.0,38.0,1.0,0.0,...,891589e7,1d1eb838,a458ea53,840e0f0f,78e2e389,bcdee96c,d16bdc4c,e8b83407,095e16c8,45840612


In [ ]:
for index in twoMIndeces['Index']:
    clear_output()
    print(index)
    if allData[allData['Index'] == index].shape[0] != 1:
        print("not good shape")
        break

In [47]:
curData = pd.read_table('dac/train.txt',skiprows = 8000000 - 1, nrows = 1,header=None)
curData.index = [8000000]
curData['Index'] = curData.index

In [49]:
allData = pd.concat([allData,curData])

In [56]:
allData.shape[0] == twoMIndeces.shape[0]

True

In [54]:
allData.to_csv('2MDataFinalDesktop.csv')